In [15]:
#pyTorch

import azureml.core
print(azureml.core.VERSION)
from azureml.core.workspace import Workspace
ws = Workspace.from_config()
ws.get_details()

from azureml.core import Environment

curated_env = Environment.get(workspace=ws, name="abi_pytorch2")



1.51.0


In [13]:
%pip install torch


Note: you may need to restart the kernel to use updated packages.


https://towardsdatascience.com/lets-do-neural-networks-d849d80fd012

In [3]:
import numpy
import pandas
import matplotlib.pyplot
import csv
import torch
import math
from sklearn.model_selection import train_test_split

Load data, make into tensors...

In [4]:
schools = pandas.read_csv("/home/azureuser/cloudfiles/code/Users/ariley/Data/Schools/School_Counts/ceara_school_count_data.csv")

schools = schools.iloc[:, 3:]

schools = schools.dropna()

df = schools

df.head()

,NUMPOINTS,ID,land,pop,built_s,built_v,smod,x,y
0,0,1.0,1000000.0,0.000000,0,0,11,-4.146928e+06,-415647.743962
1,0,2.0,1000000.0,0.000000,0,0,11,-4.146995e+06,-416168.514849
2,0,3.0,1000000.0,89.137142,4644,11618,12,-4.145898e+06,-412699.018444
3,0,4.0,1000000.0,8.601842,359,898,11,-4.146006e+06,-413307.519153
4,0,5.0,1000000.0,2.825484,141,353,11,-4.146103e+06,-414290.948276


In [5]:
# split out the validation set
train_x, val_x, train_y, val_y,train_evy,val_evy = train_test_split(
    df.drop(labels = ['NUMPOINTS','pop'],axis = 1),
    df['NUMPOINTS'],
    df['pop'],
    test_size = 0.15,
    random_state = 0,
    shuffle = True
)

# now split train and test
train_x, test_x, train_y, test_y,train_evy,test_evy = train_test_split(
    train_x,
    train_y,
    train_evy,
    test_size = 0.2,
    random_state = 0,
    shuffle = True
)

In [6]:
mapper_land = {j:str(int(j)) for j in df['land'].unique()}
mapper_smod = {j:str(int(j)) for j in df['smod'].unique()}

In [7]:
# helper function
def transform_data(data):
    
    '''
    Function to apply transformations to input data.
    '''
    
    d = data.copy()
    
    d['land'] = d['land'].map(mapper_land).astype(str)
    d['smod'] = d['smod'].map(mapper_smod).astype(str)
    
    return d

# apply groupings and transformations
train_x = transform_data(train_x)

train_x.head()

,ID,land,built_s,built_v,smod,x,y
118929,118930.0,903084,0,0,11,-3.881370e+06,-945241.256537
33969,33970.0,744911,1431,3579,11,-4.038370e+06,-731241.256537
84078,84079.0,1000000,1812,4533,11,-3.946370e+06,-718241.256537
3072,3073.0,1000000,0,0,11,-4.117370e+06,-575241.256537
145099,145100.0,999899,15215,38043,12,-3.805370e+06,-627241.256537


In [18]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        
        # linear
        self.F_hidden_one = torch.nn.Linear(75 + 1, 250)
        self.F_hidden_two = torch.nn.Linear(250, 250)
        self.F_output = torch.nn.Linear(250, 1)
        
        # add in drop out
        self.dropout_one = torch.nn.Dropout(p = 0.25)
        self.dropout_two = torch.nn.Dropout(p = 0.25)
        
        # initialise weights     
        # He initialisation
        torch.nn.init.kaiming_uniform_(self.F_hidden_one.weight)
        torch.nn.init.kaiming_uniform_(self.F_hidden_two.weight)
        torch.nn.init.kaiming_uniform_(self.F_output.weight)
        
        # initialise the final bias
        torch.nn.init.constant_(self.F_output.bias,y_hat)
        
    def forward(self, x):
        # ELU activations
        elu = torch.nn.ELU(alpha = 1)
        
        # calculate F
        F = self.dropout_one(x)
        F = self.F_hidden_one(F)
        F = elu(F)
        F = self.dropout_two(F)
        F = self.F_hidden_two(F)
        F = elu(F)
        F = self.F_output(F)
        F = torch.exp(F)
              
        return F

In [16]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.hid1 = torch.nn.Linear(4, 10)  # 4-(10-10)-1
    self.hid2 = torch.nn.Linear(10, 10)
    self.oupt = torch.nn.Linear(10, 1)

    torch.nn.init.xavier_uniform_(self.hid1.weight)
    torch.nn.init.zeros_(self.hid1.bias)
    torch.nn.init.xavier_uniform_(self.hid2.weight)
    torch.nn.init.zeros_(self.hid2.bias)
    torch.nn.init.xavier_uniform_(self.oupt.weight)
    torch.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = torch.tanh(self.hid1(x))
    z = torch.tanh(self.hid2(z))
    z = self.oupt(z)  # no activation
    return z

In [19]:
y_hat = numpy.log(df.NUMPOINTS)

# new model
model = NeuralNetwork()#.to(device)

# optimiser
optimiser = torch.optim.Adam(model.parameters(),lr = 0.001)

# training loop
for epoch in range(1,n_epoch+1):
    
    # training mode
    model.train()

    # prediction error
    F = model(train_X)
    
    # back-prop & weight update
    train_loss = poisson_loss(train_y,train_evy.float() * F)
    train_loss.backward()
    optimiser.zero_grad()
    optimiser.step()

TypeError: fill_() received an invalid combination of arguments - got (Series), but expected one of:
 * (Tensor value)
      didn't match because some of the arguments have invalid types: (!Series!)
 * (Number value)
      didn't match because some of the arguments have invalid types: (!Series!)
